# CW_Preprocessing

## 1.2 Store Cleaned Data

The following modules are imported to aid in creating, analysing and modifying databases.

In [1]:
# Import modules
import numpy as np
import pandas as pd
import sqlite3

## Task description

Your initial task is to write a Python program to create SQLite database tables for the tracking system. The program needs to clean the given attendance records, format them into the appropriate form and store them into the tables.
Save the program as CW_Preprocessing.ipynb.

The detail of the task:

• Read data from each CSV file and write them into one separate
DataFrame (e.g., dfCOA122 )

• Utilize the column headings of the DataFrame and store the details of each
session into a new DataFrame (e.g., dfCOA122Sessions).

• Create a copy of the original DataFrames to keep cleaned data (e.g.,
dfCleanCOA122).

o Rename columns of the DataFrames so that their names are short
and each column has a unique numeric session id.

o If the record of a student in the DataFrame has only null values,
then delete their information in the DataFrame.

o If any column of the DataFrame has only null values, then delete
the whole column.

o Replace each “Ex” value with null value in the DataFrame and
replace “GPS” and “X” values with the Boolean True and False
values respectively.

• Use SQLite to create a database and save it as CWDatabase.db.

• Programmatically create SQL tables in the database and store generated
DataFrames into those tables. You could use a basic data model, which
creates a database table for each DataFrame. Make sure that the tables
have the correct data types.

## Read attendance data csv file

In [2]:
def read_csv_to_df(csv_file):
    """
    This function reads a csv file and returns a pandas DataFrame.
    
    Args:
    csv_file (str): File path of the csv file.
    
    Returns:
    df: Pandas DataFrame containing csv file contents.
    """
    df = pd.read_csv(csv_file)
    return df

##  Creating dfSessions table

In [3]:
def create_sessions_df(df):
    """
    This function creates a new DataFrame 'dfSessions' containing session
    details extracted from the given DataFrame. 
    It then carries out several cleaning tasks such as column splitting,
    renaming columns,as well as setting an index.
    
    Args:
    df (pd.DataFrame): Original DataFrame from which session details are
    extracted.
    
    Returns:
    dfSessions: Pandas DataFrame containing session details.
    """
    # create a new dataframe with a column reading column headers from the csv_file
    dfSessions = pd.DataFrame(df.columns, columns = ["SessionDetails"])

    # split the SessionDetails column on the '\n' character
    split_df = dfSessions['SessionDetails'].str.split('\n', expand=True)

    # specify the names for the new columns
    split_df.columns = ['Week', 'Day', 'Time', 'Lesson', 'Room', 'Excluded']

    # update the dfSessions dataframe with the split columns
    dfSessions = dfSessions.join(split_df)

    # drop the original SessionDetails column
    dfSessions = dfSessions.drop('SessionDetails', axis=1)

    # drop the first row which shows studentID
    dfSessions = dfSessions.drop(0).reset_index(drop = True)
    
    # Extracting the semester and week values from the 'Week' column
    dfSessions[['Semester','Week']] = dfSessions['Week'].str.extract('(S\d)\.(W\d+)',
                                                                     expand = True)
    
    # Create a session ID column to set as index
    dfSessions['s_id'] = range(1, dfSessions.shape[0] + 1)
    
    # set index to Week
    dfSessions = dfSessions.set_index('s_id')
    return dfSessions

## Creating dfClean copy of the DataFrame

In [4]:
def create_dfClean(df):
    """
    This function creates a new DataFrame 'dfClean' which is a cleaned
    version of the original DataFrame. It does cleaning operations such
    as renaming columns, setting an index, removing null rows and columns,
    and giving data points Boolean values.
    
    Args:
    df (pd.DataFrame): Original DataFrame from which session details are
    extracted.
    
    Returns:
    dfClean: Cleaned Pandas DataFrame containing True, False, None values.
    """
    dfClean = df.copy()
    
    #rename first column header to s_id
    dfClean.rename(columns={dfClean.columns[0]: 's_id'}, inplace=True)

    # set index to student ID
    dfClean = dfClean.set_index('s_id')

    # rename columns to short numeric session ID
    dfClean.columns = range(1, len(dfClean.columns) + 1)

    # Delete students (rows) with only null values
    dfClean = dfClean.dropna(subset = dfClean.columns, how = "all", axis = 0)

    # Delete sessions (columns) with only null values
    dfClean = dfClean.dropna(how = "all", axis = 1)

    # Replace "Ex" with null, "GPS" and "X" with True and False
    dfClean = dfClean.replace({"Ex":None, "GPS":True, "X":False})
    return dfClean

## Creating a sqlite database & saving dataframes to sql

In [5]:
def save_to_sqlite_db(csv_file, dfClean, dfSessions):
    """
    This function creates and connects to an SQLite database, and returns
    the dataframes for both attendance and sessions tables.
    It uses a unique identifier and naming convention so that every new
    table read into the sqlite database will be saved separately under
    a new name. 
    
    Args:
    csv_file (str): File path of the CSV file.
    dfClean (pd.DataFrame): Cleaned DataFrame containing attendance data.
    dfSessions (pd.DataFrame): Cleaned DataFrame containing session details.
    
    Returns:
    df_attendance, df_sessions (SQLite tables): Cleaned SQLite dataframes
    containing 0,1,NaN for False,True,Na values in dfClean.
    """
    conn = sqlite3.connect("CWDatabase.db")
    
    # extract the unique identifier from the file name
    identifier = csv_file.split("/")[-1].split(".")[0]
    
    # name the table using the identifiers
    attendance_table = identifier + "_attendance"
    sessions_table = identifier + "_sessions"

    # Create tables in database
    dfClean.to_sql(attendance_table, conn, if_exists="replace")
    dfSessions.to_sql(sessions_table, conn, if_exists="replace")

    # show the data in the 'attendance' table
    df_attendance = pd.read_sql(f'SELECT * FROM [{attendance_table}]', conn,
                                index_col = ['s_id'])

    # show the data in the 'sessions' table
    df_sessions = pd.read_sql(f'SELECT * FROM [{sessions_table}]', conn,
                              index_col = ['s_id'])
    conn.close()
    return df_attendance, df_sessions

# Main Code

In [6]:
def main_preprocessing(csv_file):
    """
    This function is the main function that runs all the previous functions.
    
    Args:
    csv_file (str): File path of the CSV file.

    Returns:
    None
    """
    # csv_file = ../data_files/attendance_folder/22COA122ModuleRegister.csv
    # csv_file = ../data_files/attendance_folder/22COA111ModuleRegister.csv
    df = read_csv_to_df(csv_file)
    dfSessions = create_sessions_df(df)
    dfClean = create_dfClean(df)
    df_attendance, df_sessions = save_to_sqlite_db(csv_file, dfClean, dfSessions)
    display(df_attendance)
    display(df_sessions)

## Main Test Code 1: 22COA122

The output of the main code shows the cleaned dataframes saved into the SQLite database for both dfClean (or df_attendance) and dfSessions (or df_sessions).

This test code shows these two dataframes for module code 22COA122.

In [7]:
# define csv_file with correct pathway
csv_file = '../data_files/attendance_folder/22COA122ModuleRegister.csv'

main_preprocessing(csv_file)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,17,18,19
s_id,,,,,,,,,,,,,,,,,
0,1.0,1.0,NaN,1.0,0.0,NaN,0.0,0.0,NaN,1.0,0.0,NaN,0.0,0.0,1.0,NaN,0.0
1,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0
2,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,0.0,NaN,0.0,0.0,1.0,NaN,1.0
3,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,1.0,0.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0
4,1.0,1.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.0,0.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,0.0,NaN,1.0,1.0,1.0,NaN,1.0
239,1.0,1.0,NaN,1.0,1.0,NaN,1.0,0.0,NaN,0.0,1.0,NaN,1.0,1.0,1.0,NaN,0.0
240,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0


,Week,Day,Time,Lesson,Room,Excluded,Semester
s_id,,,,,,,
1,W1,Monday (03-10-2022),14:00 - 16:00,Lecture,CC012,,S1
2,W1,Thursday (06-10-2022),09:00 - 11:00,Computer Lab,N001...,,S1
3,W1,Thursday (06-10-2022),11:00 - 13:00,Computer Lab,N001...,,S1
4,W2,Monday (10-10-2022),14:00 - 16:00,Lecture,CC012,,S1
5,W2,Thursday (13-10-2022),09:00 - 11:00,Computer Lab,N001...,,S1
6,W2,Thursday (13-10-2022),11:00 - 13:00,Computer Lab,N001...,,S1
7,W3,Monday (17-10-2022),14:00 - 16:00,Lecture,CC012,,S1
8,W3,Thursday (20-10-2022),09:00 - 11:00,Computer Lab,N001...,,S1
9,W3,Thursday (20-10-2022),11:00 - 13:00,Computer Lab,N001...,,S1


## Main Test Code 2: 22COA111

The output of the main code shows the cleaned dataframes saved into the SQLite database for both dfClean (or df_attendance) and dfSessions (or df_sessions).

This test code shows these two dataframes for module code 22COA111.

In [8]:
# define csv_file with correct pathway
csv_file = '../data_files/attendance_folder/22COA111ModuleRegister.csv'

main_preprocessing(csv_file)

,1,2,3,4,5,6,8,9,11,12,14
s_id,,,,,,,,,,,
0,1.0,1.0,0.0,None,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,1.0,1.0,1.0,None,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,None,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,NaN,NaN,1.0,None,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,None,1.0,0.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
238,0.0,0.0,1.0,None,1.0,0.0,1.0,1.0,1.0,1.0,1.0
239,1.0,0.0,1.0,None,1.0,1.0,1.0,1.0,1.0,1.0,0.0
240,1.0,1.0,1.0,None,1.0,0.0,1.0,1.0,1.0,1.0,1.0


,Week,Day,Time,Lesson,Room,Excluded,Semester
s_id,,,,,,,
1,W1,Monday (03-10-2022),09:00 - 10:00,Lecture,U020,,S1
2,W1,Wednesday (05-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
3,W2,Wednesday (12-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
4,W2,Wednesday (12-10-2022),12:00 - 13:00,Tutorial,,,S1
5,W2,Thursday (13-10-2022),14:00 - 15:00,Lecture,CC011,,S1
6,W3,Wednesday (19-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
7,W3,Wednesday (19-10-2022),12:00 - 13:00,Tutorial,,,S1
8,W3,Thursday (20-10-2022),14:00 - 15:00,Lecture,CC011,,S1
9,W4,Wednesday (26-10-2022),10:00 - 11:00,Lecture,SMB014,,S1


## Test Code for Firat to enter a new module:

The output of the main code shows the cleaned dataframes saved into the SQLite database for both dfClean (or df_attendance) and dfSessions (or df_sessions).

This test code shows these two dataframes for the new module code you would like to enter.

In [9]:
# define csv_file with correct pathway
path = "../data_files/attendance_folder/"
file = input("enter file (e.g. 22COA111ModuleRegister.csv):")
csv_file = path+file

#call main code function
main_preprocessing(csv_file)

enter file (e.g. 22COA111ModuleRegister.csv):22COA111ModuleRegister.csv


,1,2,3,4,5,6,8,9,11,12,14
s_id,,,,,,,,,,,
0,1.0,1.0,0.0,None,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,1.0,1.0,1.0,None,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,None,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,NaN,NaN,1.0,None,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,None,1.0,0.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
238,0.0,0.0,1.0,None,1.0,0.0,1.0,1.0,1.0,1.0,1.0
239,1.0,0.0,1.0,None,1.0,1.0,1.0,1.0,1.0,1.0,0.0
240,1.0,1.0,1.0,None,1.0,0.0,1.0,1.0,1.0,1.0,1.0


,Week,Day,Time,Lesson,Room,Excluded,Semester
s_id,,,,,,,
1,W1,Monday (03-10-2022),09:00 - 10:00,Lecture,U020,,S1
2,W1,Wednesday (05-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
3,W2,Wednesday (12-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
4,W2,Wednesday (12-10-2022),12:00 - 13:00,Tutorial,,,S1
5,W2,Thursday (13-10-2022),14:00 - 15:00,Lecture,CC011,,S1
6,W3,Wednesday (19-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
7,W3,Wednesday (19-10-2022),12:00 - 13:00,Tutorial,,,S1
8,W3,Thursday (20-10-2022),14:00 - 15:00,Lecture,CC011,,S1
9,W4,Wednesday (26-10-2022),10:00 - 11:00,Lecture,SMB014,,S1


# Testing all functions in the Notebook

Running the function (read_csv_to_df) with any csv file shows that it successfully reads the csv file and presents it as a pandas dataframe.

This dataframe shows the raw uncleaned data.

In [10]:
csv_file = '../data_files/attendance_folder/22COA111ModuleRegister.csv'
df = read_csv_to_df(csv_file)
display(df)

,sid_x,S1.W1\nMonday (03-10-2022)\n09:00 - 10:00\nLecture\nU020\n,S1.W1\nWednesday (05-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W2\nWednesday (12-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W2\nWednesday (12-10-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W2\nThursday (13-10-2022)\n14:00 - 15:00\nLecture\nCC011\n,S1.W3\nWednesday (19-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W3\nWednesday (19-10-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W3\nThursday (20-10-2022)\n14:00 - 15:00\nLecture\nCC011\n,S1.W4\nWednesday (26-10-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W4\nWednesday (26-10-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W4\nThursday (27-10-2022)\n14:00 - 15:00\nLecture\nCC011\n,S1.W5\nWednesday (02-11-2022)\n10:00 - 11:00\nLecture\nSMB014\n,S1.W5\nWednesday (02-11-2022)\n12:00 - 13:00\nTutorial\n\n,S1.W5\nThursday (03-11-2022)\n14:00 - 15:00\nLecture\nCC011\n
0,0,GPS,GPS,X,NaN,GPS,GPS,NaN,GPS,GPS,NaN,X,X,NaN,X
1,1,GPS,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS
2,2,GPS,GPS,GPS,NaN,X,X,NaN,GPS,X,NaN,GPS,X,NaN,X
3,3,NaN,NaN,GPS,NaN,GPS,X,NaN,GPS,X,NaN,GPS,X,NaN,GPS
4,4,GPS,X,X,NaN,GPS,X,NaN,GPS,X,NaN,GPS,X,NaN,GPS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,238,X,X,GPS,NaN,GPS,X,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS
232,239,GPS,X,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,X
233,240,GPS,GPS,GPS,NaN,GPS,X,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS
234,241,GPS,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS,GPS,NaN,GPS


Running the function (create_sessions_df) shows the newly created sessions table which contains session details for every session in the semester.

In [11]:
dfSessions = create_sessions_df(df)
display(dfSessions)

,Week,Day,Time,Lesson,Room,Excluded,Semester
s_id,,,,,,,
1,W1,Monday (03-10-2022),09:00 - 10:00,Lecture,U020,,S1
2,W1,Wednesday (05-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
3,W2,Wednesday (12-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
4,W2,Wednesday (12-10-2022),12:00 - 13:00,Tutorial,,,S1
5,W2,Thursday (13-10-2022),14:00 - 15:00,Lecture,CC011,,S1
6,W3,Wednesday (19-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
7,W3,Wednesday (19-10-2022),12:00 - 13:00,Tutorial,,,S1
8,W3,Thursday (20-10-2022),14:00 - 15:00,Lecture,CC011,,S1
9,W4,Wednesday (26-10-2022),10:00 - 11:00,Lecture,SMB014,,S1


Running the function (create_dfClean) shows the cleaned version of the initial raw data with all the modifications after the cleaning process.

In [12]:
dfClean = create_dfClean(df)
display(dfClean)

,1,2,3,4,5,6,8,9,11,12,14
s_id,,,,,,,,,,,
0,True,True,False,NaN,True,True,True,True,False,False,False
1,True,True,True,NaN,True,True,True,True,True,True,True
2,True,True,True,NaN,False,False,True,False,True,False,False
3,NaN,NaN,True,NaN,True,False,True,False,True,False,True
4,True,False,False,NaN,True,False,True,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...
238,False,False,True,NaN,True,False,True,True,True,True,True
239,True,False,True,NaN,True,True,True,True,True,True,False
240,True,True,True,NaN,True,False,True,True,True,True,True


Running the function (save_to_sqlite_db) shows that the cleaned dataframe and the session details dataframes are saved into the SQLite database 'CWDatabase.db' as the function also reads the sql versions of these dataframes to verify that the dataframe has been successfully saved.

In [13]:
df_attendance, df_sessions = save_to_sqlite_db(csv_file, dfClean, dfSessions)
display(df_attendance)
display(df_sessions)

,1,2,3,4,5,6,8,9,11,12,14
s_id,,,,,,,,,,,
0,1.0,1.0,0.0,None,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,1.0,1.0,1.0,None,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,None,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,NaN,NaN,1.0,None,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,None,1.0,0.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
238,0.0,0.0,1.0,None,1.0,0.0,1.0,1.0,1.0,1.0,1.0
239,1.0,0.0,1.0,None,1.0,1.0,1.0,1.0,1.0,1.0,0.0
240,1.0,1.0,1.0,None,1.0,0.0,1.0,1.0,1.0,1.0,1.0


,Week,Day,Time,Lesson,Room,Excluded,Semester
s_id,,,,,,,
1,W1,Monday (03-10-2022),09:00 - 10:00,Lecture,U020,,S1
2,W1,Wednesday (05-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
3,W2,Wednesday (12-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
4,W2,Wednesday (12-10-2022),12:00 - 13:00,Tutorial,,,S1
5,W2,Thursday (13-10-2022),14:00 - 15:00,Lecture,CC011,,S1
6,W3,Wednesday (19-10-2022),10:00 - 11:00,Lecture,SMB014,,S1
7,W3,Wednesday (19-10-2022),12:00 - 13:00,Tutorial,,,S1
8,W3,Thursday (20-10-2022),14:00 - 15:00,Lecture,CC011,,S1
9,W4,Wednesday (26-10-2022),10:00 - 11:00,Lecture,SMB014,,S1


## See list of sqlite tables in CWDatabase

Below is a simple list of SQLite tables saved into the database created. This is just to remember the naming format of each file so it is easier to refer to names in the future tasks

In [14]:
### Check list of sqlite tables saved in database
conn = sqlite3.connect('CWDatabase.db')

# Getting all tables from sqlite_master
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
 
# Creating cursor object using connection object
cursor = conn.cursor()
     
# executing sql query
cursor.execute(sql_query)
    
# printing all tables list
print(cursor.fetchall())

#close connection
conn.close()

[('22COA122ModuleRegister_attendance',), ('22COA122ModuleRegister_sessions',), ('22COA111ModuleRegister_attendance',), ('22COA111ModuleRegister_sessions',)]
